# Description

This notebook is used to extract ET and other data for analysis in other notebooks/code.

The SEBAL calculations for ET are leveraged from https://github.com/gee-hydro/geeSEBAL

Usage:

  1) Show map with labeled samples for rain or irrigated locations

  2) Manually specify Area of Interest (AOI) with bounding box on map

  3) Specify the filename and number of sample locations to generate within the AOI

  4) Launch batch data extraction


# Setup Notebook

In [1]:
#!pip install geemap
#!pip install geopandas

In [2]:
import ee
#ee.Authenticate()
ee.Initialize()
import geemap

import numpy as np
import pandas as pd
import datetime
from timeit import default_timer as timer
from datetime import timedelta

In [3]:
%cd /content
#!git clone https://github.com/gee-hydro/geeSEBAL.git
!git clone https://github.com/earthdataanalytics/irrigation.git
#!cp -r geeSEBAL/etbrasil/ irrigation/
%cd irrigation/

/content
fatal: destination path 'irrigation' already exists and is not an empty directory.
/content/irrigation


### Import Custom Libraries

In [4]:
from pipeline import boundaries as bnd
from pipeline import rainfed_labels as rain_labels
from pipeline import labeledsamples as ls
from pipeline import dataextract as dx
from pipeline import cropmasks as msk

## 1) Show labeled locations on map

Rain = Red

Irrigated = Blue

In [5]:
# irrMapper and LANID data
irr_west, rain_west, irr_east, rain_east = ls.getRawLabeledData()

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [6]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.addLayer(irr_west, { 'color': 'blue'}, 'irrigated irrM west', opacity=0.5)
Map.addLayer(rain_west, { 'color': 'red'}, 'rainfed irrM west', opacity=0.5)
#Map.addLayer(irr_east, { 'color': 'blue'}, 'irrigated LANID east', opacity=0.5)
#Map.addLayer(rain_east, { 'color': 'red'}, 'rainfed LANID east', opacity=0.5)
Map.addLayer(rain_labels.manual_rainfed, { 'color': 'red'}, 'rainfed manual labels', opacity=0.5)
Map.center_object(irr_west)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## 2) Draw a bounding box on the map around the area in which to extract data

## 3) Generate sample locations within the bounding box

Specify the:

* name of the AOI
* number of sample locations to generate


This displays the generated samples as Yellow points on the map.

In [7]:
# specify manually
aoi_name = 'test_throwaway' # used in the saved filename
num_samples = 50 # number of samples locations for each class

if Map and (len(Map.draw_features) > 0):
    AOI = ee.FeatureCollection(Map.draw_features)
    sample_locs = ls.generateAnnualSampleLocations(AOI, aoi_name, 
                                              num_samples=num_samples,
                                              #start_yr=2015, end_yr=2021,
                                            )
    print('Number of sample locations generated by type', sample_locs.aggregate_histogram('POINT_TYPE').getInfo())
    Map.addLayer(sample_locs.draw(color="yellow", pointRadius=5, strokeWidth=3), {}, 'sample_locations')
    Map.centerObject(sample_locs)

else:
    print("No AOI selected on Map")

Number of sample locations generated by type {'0': 1, '1': 1}


## 4) Launch batch data extraction

In [8]:
if sample_locs:
    et = dx.extractData(sample_locs, aoi_name)

Number of locations to extract = 2


 17%|█▋        | 1/6 [01:30<07:30, 90.14s/it]


KeyboardInterrupt: ignored

In [ ]:
!earthengine task list --status RUNNING READY | wc -l

0
